# Training Loop

1. Retrieve the [CheXpert](https://stanfordmlgroup.github.io/competitions/chexpert) dataset. The validation dataset can be found [here](https://stanfordaimi.azurewebsites.net/datasets/23c56a0d-15de-405b-87c8-99c30138950c) contained within the CheXpert directory. Note: the test dataset provided by the researchers does not contain the same structured labels (ie: `Age`, `Sex`, etc..) as the train and validation set. We perform random sampling from the train set to retrieve the test set for this project. 
2. Generate the test set by sampling from the training set. In our case, we randomly selected 2% of the training data for testing.
3. Create the dictionaries needed to read each dataset (train, valid, test) and save them to disk. Since the training data was much too large, we create 20 separate training datasets that we iterate through when training.

See the `README.md` file in the data directory for data setup specifics.

4. Model checkpoints are made each epoch during training. Train loss, validation loss, mean AUROC during training and mean AUROC during validation are saved to a csv file each iteration.
5. The Model is trained on 20 training datasets, each representing a randomly selected 5% (without replacement) of the entire set for 10 epochs each. The batch size used was 24.


# IRENE Results

In [1]:
import pandas as pd
df_irene = pd.read_csv('checkpoints/metrics.csv')
df_irene

,dataset,set_num,epoch,trn_loss,trn_mean_auroc,val_loss,val_mean_auroc
0,./data/actual/train_0.pkl,0,1,0.563882,0.591154,0.476869,0.574488
1,./data/actual/train_0.pkl,0,2,0.548167,0.635756,0.462459,0.586174
2,./data/actual/train_0.pkl,0,3,0.539093,0.655897,0.458620,0.660027
3,./data/actual/train_0.pkl,0,4,0.525229,0.678167,0.437948,0.680434
4,./data/actual/train_0.pkl,0,5,0.519875,0.686688,0.433910,0.689004
...,...,...,...,...,...,...,...
195,./data/actual/train_19.pkl,19,6,0.406080,0.836532,0.407489,0.826824
196,./data/actual/train_19.pkl,19,7,0.403281,0.839203,0.407128,0.821158
197,./data/actual/train_19.pkl,19,8,0.401231,0.841086,0.408375,0.828896
198,./data/actual/train_19.pkl,19,9,0.399257,0.842605,0.410504,0.819801


Plotting the train and validation loss per each training iteration. Each of the 20 sets were trained for 10 epochs before moving on to the next.

Selecting the best model based on validation loss.

In [2]:
best_mod = df_irene[df_irene['val_loss'] == df_irene['val_loss'].min()]
best_mod

,dataset,set_num,epoch,trn_loss,trn_mean_auroc,val_loss,val_mean_auroc
160,./data/actual/train_16.pkl,16,1,0.439726,0.805184,0.36235,0.811331


The best model based on validation loss corresponds to the model saved as `set_16_epc_1.pt`.

### Predicting on Test Set